In [5]:
import pandas as pd
data = pd.read_csv("/content/drive/MyDrive/DATA/P10/billets_production (1).csv")
test_billets = pd.read_csv("/content/drive/MyDrive/DATA/billets_test (2).csv")

In [2]:
def detect(data):
  # pack 
  import pandas as pd
  import numpy as np
  from sklearn.preprocessing import StandardScaler
  from sklearn.linear_model import LogisticRegression
  from sklearn.metrics import confusion_matrix
  from sklearn.cluster import KMeans
  # DATA entrainement
  df = pd.read_csv("/content/drive/MyDrive/DATA/P10/billet_clean.csv", delimiter=",")
  from sklearn.model_selection import train_test_split
  X = df.drop("is_genuine", axis="columns")
  X = X.drop("Unnamed: 0", axis="columns")
  X = X.values
  y = df.is_genuine
  Y = y.values
  Y = Y.reshape(len(Y),1)
  X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)
  sc = StandardScaler()
  X_train = sc.fit_transform(X_train)
  X_test = sc.transform(X_test)
  # Train LogisticRegression model
  log = LogisticRegression()
  log.fit(X_train,y_train)
  log_score = log.score(X_test,y_test)
  # Train KMeans model
  km = KMeans(n_clusters=2, init="k-means++")
  km.fit_predict(X_train,y_train)
  y_pred_km = km.predict(X_test)
  tst = pd.DataFrame(y_test)
  prd = pd.DataFrame(y_pred_km)
  c = tst.merge(prd, left_index=True, right_index=True)
  c["correct"]= c["0_x"] == c["0_y"]
  import collections
  nbTrue = collections.Counter(c.correct)[True]
  nbFalse = collections.Counter(c.correct)[False]
  # Permutation
  inversion = 0
  if nbFalse > nbTrue:
    inversion = 1
    for i in range(len(y_pred_km)):
      if y_pred_km[i]==0:
        y_pred_km[i] = 1
      else:
        y_pred_km[i] =0
  tst = pd.DataFrame(y_test)
  prd = pd.DataFrame(y_pred_km)
  c = tst.merge(prd, left_index=True, right_index=True)
  c["correct"]= c["0_x"] == c["0_y"]
  nbTrue = collections.Counter(c.correct)[True]
  km_score = nbTrue/len(c)
  km_score
  print("log score ",log_score)
  print("km score ",km_score)
  print("is permuted ",bool(inversion))
  data.set_index("id", inplace=True)
  data
  X = data.values
  X = sc.transform(X)
  log_prod = log.predict(X)
  km_prod = km.predict(X)
  if inversion == 1:
    for i in range(len(km_prod)):
      if km_prod[i]==0:
        km_prod[i] = 1
      else:
        km_prod[i] =0
  km_prod = [bool(item) for item in km_prod]
  data["log_predict"] = log_prod
  data["KMeans_predict"] = km_prod
  return(data)


In [3]:
detect(data)

log score  0.99
km score  0.98
is permuted  True


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


,diagonal,height_left,height_right,margin_low,margin_up,length,log_predict,KMeans_predict
id,,,,,,,,
A_1,171.76,104.01,103.54,5.21,3.30,111.42,False,False
A_2,171.87,104.17,104.13,6.00,3.31,112.09,False,False
A_3,172.00,104.58,104.29,4.99,3.39,111.57,False,False
A_4,172.49,104.55,104.34,4.44,3.03,113.20,True,True
A_5,171.65,103.63,103.56,3.77,3.16,113.33,True,True


In [6]:
detect(test_billets)

log score  0.9966666666666667
km score  0.99
is permuted  True


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


,diagonal,height_left,height_right,margin_low,margin_up,length,log_predict,KMeans_predict
id,,,,,,,,
B_1,172.09,103.95,103.73,4.39,3.09,113.19,True,True
B_2,171.52,104.17,104.03,5.27,3.16,111.82,False,False
B_3,171.78,103.80,103.75,3.81,3.24,113.39,True,True
B_4,172.02,104.08,103.99,5.57,3.30,111.10,False,False
B_5,171.79,104.34,104.37,5.00,3.07,111.87,False,False
